# CARE demo

This notebook covers the essential steps to generate and manipulate the chemical reaction networks (CRNs) generated with CARE. All the steps shown here are included in the main script `care.scripts.care_run`, which is the recommended way to generate the CRN.
Install Ipykernel to run the notebook!

## Generate Chemical Reaction Network: Blueprint

To generate the CRN blueprint, i.e. the listing of the potential species and elementary reactions in your CRN, use the function `gen_blueprint`.

In [ ]:
from care.crn.utils.blueprint import gen_blueprint

### Inputs

*ncc* = Network Carbon Cutoff, i.e. the maximum number of C atoms that a species in the CRN can possess  <br>
*noc* = Network Oxygen Cutoff, i.e. the maximum number of C atoms that a species in the CRN can possess  <br>
*cyclic* = Whether species with rings are should be included or not (True/False)  <br>
*additional_rxns* = Whether to include intramolecular rearrengments (True/False)  <br>
*electro* = Whether to generate a CRN in electrochemical conditions

In [ ]:
ncc, noc = 2, 2
cyclic = False
rearr_rxns = True
electro = False

inters, rxns = gen_blueprint(ncc=ncc, 
                             noc=noc, 
                             cyclic=cyclic, 
                             additional_rxns=rearr_rxns, 
                             electro=electro)

The output of the function is a dict of `Intermediate` objects indexed by their InChIKey (plus a phase-identifier character, '*' for adsorbed, 'g' for gas-phase), and a list of `ElementaryReaction` instances.

In [ ]:
print(type(inters)) 
print(type(inters[list(inters.keys())[0]]))
print(type(rxns))
print(type(rxns[100]))

### ElementaryReaction

In [ ]:
r = rxns[170]
print(r)
print(type(r))
print(r.repr_hr)  # human-readable text representation
print(r.r_type)  # bond-breaking type
print(r.components)  # components of the reaction
print(r.stoic)  # Soichiometry coefficients in the reaction
print(r.e_rxn)  # reaction energy
print(r.e_act)  # reaction activation energy

Observations:
- species participating in the elementary reactions are displayed as '[n]InChIKeyP(formula)', with 'n' being the absolute stoichiometric coefficient, formula being the brute formula provided by ASE, and 'P' after the InChIKey being a phase-identifier character, '*' for adsorbed species and 'g' for gaseous species. <br>
- ``e_rxn`` and ``e_act`` are ``None`` as these reaction properties still have to be evaluated


### Intermediate

In [ ]:
a = inters['WSFSSNUMVMOOMR-UHFFFAOYSA-N*']
print(a)
print(type(a))
print(a.phase)
print(a.smiles)
print(a.ref_energy())
print(a.is_closed_shell())
print(a['C'])  # number of carbon atoms
print(a['H'])  # number of hydrogen atoms
print(a['O'])  # number of oxygen atoms
print(a.ads_configs)  # adsorption configurations

Observations:
- ``Intermediate.ads_configs`` is a dictionary where the adsorption configurations will be stored. As the blueprint considers the network generation in gas-phase, is empty for now.
- the ``Intermediate.code`` contains a phase-identifier character between the InChIKey and the brute formula, being '*' for adsorbed species and 'g' for gaseous species. Changing ``WSFSSNUMVMOOMR-UHFFFAOYSA-N*`` to ``WSFSSNUMVMOOMR-UHFFFAOYSA-Ng`` returns a copy of the same object, they will represent different species onces these are evaluated.

## Generate Chemical Reaction Network: Energy Evaluation

Now we define the surface under study and evaluate the energies of intermediates and reactions

## Load catalyst surface

In [ ]:
from care.utils import load_surface
from care import DB_PATH

In [ ]:
metal = 'Ru'
facet = '0001'  # hkl notation

surface = load_surface(DB_PATH, metal, facet)

print(surface)
print(type(surface))
print(surface.slab)  # ASE Atoms object
print(surface.active_sites)  # Obtained with ACAT

## Load Intermediate energy evaluator

In our case, this is GAME-Net-UQ, a graph neural network trained on DFT data.

In [ ]:
from care.gnn.interface import GameNetUQInter
from care import MODEL_PATH

In [ ]:
gnn = GameNetUQInter(MODEL_PATH, surface, None)

print(gnn)

## Evaluate intermediates (sequential)

We have 171 intermediates to evaluate. We will show here the sequential evaluation to give an idea. If you want to get the same output faster, skip this section and run the parallel evaluation.

In [ ]:
inters_evaluated = {}
for k, inter in inters.items():
    print(inter)
    inters_evaluated[k] = gnn.eval(inter)
    print(inters_evaluated[k].ads_configs)
    

In [ ]:
inters_evaluated['LHGAACIDYUKUTF-UHFFFAOYSA-N*'].ads_configs

## Evaluate intermediates (parallel)

Parallel execution across multiple CPU cores can speed up the evaluation process. In our case, the parallel execution took 1min 51s, while the sequential 20 minutes (10x speed up with 24 cores).

In [ ]:
import multiprocessing as mp
import resource

_, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (hard, hard))

In [ ]:
tasks = [(gnn, inter) for inter in inters.values()]
print(f'{len(tasks)} intermediates to evaluate')

In [ ]:
def mp_func(gnn, inter):
    print(inter.code)
    return gnn.eval(inter)

with mp.Pool(mp.cpu_count()) as pool:
    results = pool.starmap(mp_func, tasks)

inters_evaluated = {i.code: i for i in results}

In [ ]:
# get random key and print the adsorption configurations
key = list(inters_evaluated.keys())[0]
inters_evaluated[key].ads_configs

## Evaluate reaction properties

In [ ]:
from care.gnn.interface import GameNetUQRxn

In [ ]:
gnn_rxn = GameNetUQRxn(MODEL_PATH, inters_evaluated)
print(gnn_rxn)

print(f'{len(rxns)} reactions to evaluate')

In [ ]:
rxns_evaluated = []
for rxn in rxns:
    print(rxn)
    rxns_evaluated.append(gnn_rxn.eval(rxn))

rxns_evaluated = sorted(rxns_evaluated)

In [ ]:
import random
idx = random.randint(0, len(rxns_evaluated))

step_example = rxns_evaluated[idx]
print(step_example, step_example.r_type)
print(step_example.e_is)
print(step_example.e_ts)
print(step_example.e_fs)
print(step_example.e_rxn)
print(step_example.e_act)

## Explore the CRN

Everything is now evaluated. Let's assemble the reaction network.


In [ ]:
from care import ReactionNetwork

In [ ]:
crn = ReactionNetwork(intermediates=inters_evaluated, 
                      reactions=rxns_evaluated,
                      surface=surface,
                      ncc=ncc,
                        noc=noc,
                      type='thermal')

print(crn)

Get top-10 hubs in the network

In [ ]:
crn.get_hubs(10)

Iterate through elementary steps and show reaction energy and activation barrier. Each property is displayed as a tuple where the first item is the mean value and the second the standard deviation (uncertainty) of the prediction. Elementary reactions having same values for both reaction energy and barrier correspond to barrierless endothermic steps.

In [ ]:
for step in crn:
    print(step.repr_hr, 'DHR = {:.2f} eV, Eact = {:.2f} eV, {}'.format(step.e_rxn[0], step.e_act[0], step.__class__) )

In [ ]:
# Define column widths
repr_hr_width = max(len(step.repr_hr) for step in crn) + 2
dhr_width = 10
eact_width = 10
class_width = 55

# Print header (optional)
header = "{:<{}} {:<{}} {:<{}} {}".format("Step", repr_hr_width, "DHR (eV)", dhr_width, "Eact (eV)", eact_width, "Class")
print(header)
print("=" * (repr_hr_width + dhr_width + eact_width + class_width))

# Print each step with aligned columns
for step in crn:
    repr_hr_str = step.repr_hr.ljust(repr_hr_width)
    dhr_str = "{:.2f}".format(step.e_rxn[0]).ljust(dhr_width)
    eact_str = "{:.2f}".format(step.e_act[0]).ljust(eact_width)
    class_str = str(step.__class__).ljust(class_width)
    print(f"{repr_hr_str} {dhr_str} {eact_str} {class_str}")


Visualize elementary steps by providing the reaction index. Remember that the uncertainty of the properties is a relative measure of the model uncertainty across multiple predictions!

In [ ]:
crn.visualize_reaction(59, show_uncertainty=False)

Get number of closed-shell molecules in the network

In [ ]:
crn.num_closed_shell_mols

Visualize intermediates

In [ ]:
crn.visualize_intermediate('VRLIPUYDFBXWCH-UHFFFAOYSA-N*')

Visuliaze graph represntation

In [ ]:
from care.gnn.graph_tools import graph_plotter

adsorption_configs = crn.intermediates['VRLIPUYDFBXWCH-UHFFFAOYSA-N*'].ads_configs
adsorption_configs

In [ ]:
graph_plotter(adsorption_configs['15']['pyg'], node_index=False, dpi=150)

Select elementary reactions belonging to a specific reaction type

In [ ]:
crn.search_reaction(r_types=['adsorption'])

Select all the elementary steps involving a specific intermediate, e.g., CO2*. A current limitation is the fact that to select a specific molecule/fragment, you have to first find the corresponding code.

In [ ]:
crn.search_reaction(inters=['CURLTUGMZLYLDI-UHFFFAOYSA-N*'])

Select elementary reaction involving a specific intermediate in a specific reaction type

In [ ]:
# CO2* adsorption

crn.search_reaction(r_types=['adsorption'], inters=['CURLTUGMZLYLDI-UHFFFAOYSA-N*'])

Search for intermediates having specific stoichiometry

In [ ]:
crn.search_inter_by_elements({'C': 2, 'H': 1})

In [ ]:
# CH* C-C coupling

crn.search_reaction(r_types=['C-C'], inters=['VRLIPUYDFBXWCH-UHFFFAOYSA-N*'])

Get NetworkX graph of the reaction network

In [ ]:
crn_graph = crn.graph

print(crn_graph)

Plot crn graph with NetworkX function

In [ ]:
from networkx import draw

draw(crn_graph, with_labels=False, node_size=15, edge_color='gray')

## Manipulating elementary reactions

CRNs built with CARE contain all the possible elementary reactions that could ideally occur within a defined chemical space. From this point, two options are available: (i) directly post-processing the generated CRN by means of microkinetic modeling, or (ii) selecting specific paths and extracting a subset of steps to work with based on your intuition/ideas/etc. Here we show how to manipulate elementary reactions separately.

In [ ]:
# Get random steps

r1_idx = random.randint(0, len(crn))
r2_idx = random.randint(0, len(crn))
r3_idx = random.randint(0, len(crn))

r1 = crn[r1_idx]
r2 = crn[r2_idx]
r3 = crn[r3_idx]

print(r1, r1.e_rxn)
print(r2, r2.e_rxn)
print(r3, r3.e_rxn)


Multiply an elementary reaction by scalar value

In [ ]:
r4 = r1 * 3.14
print(r4, r4.e_rxn) 
print(type(r4))

Sum two elementary reactions

In [ ]:
r5 = r1 + r2
print(r5, r5.e_rxn)
print(type(r5))

Generate linear combination of elementary reactions

In [ ]:
r6 = 0.5 * r1 + 2 * r2 + 0.75 * r3
print(r6, r6.e_rxn)
print(type(r6))

Observations:
- Applying linear combinations to elementary reactions produces a ``ReactionMechanism`` object, which is a subclass of ``ElementaryReaction``.
- The global stoichiometry, as well as the global reaction path energy, are automatically evaluated.
- No information about the sequence order of the steps is now included within the class, will be soon included.
- One can select the desired steps and construct its own reaction mechanism starting from the pool of elementary reactions in the CRN. Selection of teh desired elementary steps is not trivial and will be improved in the future, for now make use of the ``ReactionNetwork.search_reaction()`` class method.